In [8]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

2.10.0+cu128
True
0
NVIDIA GeForce RTX 4070 Laptop GPU
12.8


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [10]:
from train_model import CustomCNN, evaluate_model, evaluate_with_tta

In [4]:
# Load the checkpoint
checkpoint = torch.load('/home/muhammad_adib/imgclass_cnn/intel_cnn_model.pth', map_location=device)

# Re-initialize the model architecture (must match the saved model)
model = CustomCNN(num_classes=len(checkpoint['class_names'])).to(device)

# Load the saved weights
model.load_state_dict(checkpoint['model_state_dict'])

# Set to evaluation mode
model.eval()

# Access saved information
print(f"Classes: {checkpoint['class_names']}")
print(f"Test accuracy: {checkpoint['test_accuracy']*100:.2f}%")

Classes: ['buildings', 'forest', 'mountain', 'sea', 'street']
Test accuracy: 95.12%


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset, DataLoader
from torchvision import datasets, transforms
import os

# Image size - must be consistent across train and validation
IMG_SIZE = 150  # Use same size for both

# 1. Define your two different sets of transforms
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),  # Same size as training!
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 2. Load the base dataset (no transform yet)
path = '/home/muhammad_adib/imgclass_cnn/Data'
base_dataset = datasets.ImageFolder(root=path)

# 3. Get stratified indices
# 70-15-15 split
train_val_idx, test_idx = train_test_split(
    np.arange(len(base_dataset.targets)),
    test_size=0.15, # 15% for test
    stratify=base_dataset.targets,
    random_state=42
)

train_idx, val_idx = train_test_split(
    train_val_idx,
    test_size=0.1765, # ~15% of original data for validation
    stratify=[base_dataset.targets[i] for i in train_val_idx],
    random_state=42
)

# 4. Create subsets and manually attach the transforms
# We use a simple wrapper class or just apply them via Subset
class ApplyTransform(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y
        
    def __len__(self):
        return len(self.subset)

train_dataset = ApplyTransform(Subset(base_dataset, train_idx), transform=train_transform)
val_dataset = ApplyTransform(Subset(base_dataset, val_idx), transform=val_transform)
test_dataset = ApplyTransform(Subset(base_dataset, test_idx), transform=val_transform)

# 5. DataLoaders - Using num_workers=0 to avoid BrokenPipeError
# For RTX 4070, GPU is fast enough that this won't be a bottleneck
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True)

# Get class names
class_names = sorted(os.listdir(path))
num_classes = len(class_names)
print(f"\nClass names: {class_names}")
print(f"Number of classes: {num_classes}")

print(f"Total images: {len(base_dataset)}")
print(f"Training set: {len(train_dataset)}")
print(f"Validation set: {len(val_dataset)}")
print(f"Test set: {len(test_dataset)}")


Class names: ['buildings', 'forest', 'mountain', 'sea', 'street']
Number of classes: 5
Total images: 14063
Training set: 9843
Validation set: 2110
Test set: 2110


In [13]:
test_accuracy = evaluate_model(model, test_loader, class_names)
print(f"\nTest Accuracy: {test_accuracy*100:.2f}%")


Testing: 100%|██████████| 33/33 [00:04<00:00,  6.72it/s]


Test Accuracy: 0.9512 (95.12%)

Classification Report:
              precision    recall  f1-score   support

   buildings       0.91      0.92      0.91       394
      forest       1.00      0.98      0.99       411
    mountain       0.94      0.98      0.96       456
         sea       0.98      0.94      0.96       416
      street       0.94      0.93      0.93       433

    accuracy                           0.95      2110
   macro avg       0.95      0.95      0.95      2110
weighted avg       0.95      0.95      0.95      2110


Test Accuracy: 95.12%


In [14]:
tta_accuracy = evaluate_with_tta(model, test_loader, device)
print(f"TTA Test Accuracy: {tta_accuracy*100:.2f}%")

Final TTA Accuracy: 95.21%
TTA Test Accuracy: 95.21%
